In [1]:
param_market = 'AU'

In [ ]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(666):
            try:
                if param_market == 'US':
                    url = 'https://www.ebay.com/itm/' + input_.loc[a, 'Item_Number'] + '?_ul=US&_stpos=91710&orig_cvip=true'
                elif param_market == 'DE':
                    url = 'https://www.ebay.de/itm/' + input_.loc[a, 'Item_Number'] + '?_ul=DE&_stpos=10115&orig_cvip=true'
                elif param_market == 'AU':
                    url = 'https://www.ebay.com.au/itm/' + input_.loc[a, 'Item_Number'] + '?_ul=AU&_stpos=2019&orig_cvip=true'
                elif param_market == 'UK':
                    url = 'https://www.ebay.co.uk/itm/' + input_.loc[a, 'Item_Number'] + '?_ul=GB&_stpos=BS81QU&orig_cvip=true'

                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script></body></html>')[0] + ']}]}')
                
                # = = = = = = = = = = = = = = =
                
                market_id = json_data['o']['g']['marketplaceId'].strip()
                
                # = = = = = = = = = = = = = = =
                
                country = json_data['o']['g']['country'].strip()

                # = = = = = = = = = = = = = = =
                
                language = json_data['o']['g']['language'].strip()

                # = = = = = = = = = = = = = = =
                
                locale = json_data['o']['g']['locale'].strip()

                # = = = = = = = = = = = = = = =
                
                item_id = json_data['o']['g']['itemId'].strip()

                # = = = = = = = = = = = = = = =
                
                if param_market == 'US':
                    if market_id != 'EBAY-US' or country != 'US' or language != 'en' or locale != 'en-US' or item_id != input_.loc[a, 'Item_Number']:
                        continue
                if param_market == 'AU':
                    if market_id != 'EBAY-AU' or country != 'AU' or language != 'en' or locale != 'en-AU' or item_id != input_.loc[a, 'Item_Number']:
                        continue
                elif param_market == 'UK':
                    if market_id != 'EBAY-GB' or country != 'GB' or language != 'en' or locale != 'en-GB' or item_id != input_.loc[a, 'Item_Number']:
                        continue
                    
                # = = = = = = = = = = = = = = =
                
                specification = json_data['o']['w'][0][2]['model']['modules']['ABOUT_THIS_ITEM']['sections']['features']['dataItems']

                # = = = = = = = = = = = = = = =
                
                if param_market in ['US', 'AU', 'UK']:
                    if 'VEHICLE_SELECTION' in json_data['o']['w'][0][2]['model']['modules']:
                        category_id = json_data['o']['w'][0][2]['model']['modules']['VEHICLE_SELECTION']['callToActions'][0]['action']['params']['categoryId'].strip()
                        # print(json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['title']['textSpans'][0]['text'].strip())
                        vehicle_page = json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['itemsPerPage']['label']['textSpans'][0]['text'].split('of')[1].strip()
                    else:
                        category_id = ''
                        vehicle_page = ''
                elif param_market == 'DE':
                    if 'MOTORS_TAB' in json_data['o']['w'][0][2]['model']['modules']:
                        category_id = json_data['o']['w'][0][2]['model']['modules']['MOTORS_TAB']['tabContent']['vehicleHsnTsnLookupTab']['callToActions'][0]['action']['params']['categoryId'].strip()
                        # print(json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['title']['textSpans'][0]['text'].strip())
                        vehicle_page = json_data['o']['w'][0][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['itemsPerPage']['label']['textSpans'][0]['text'].split('von')[1].strip()
                    else:
                        category_id = ''
                        vehicle_page = ''
                        
                # = = = = = = = = = = = = = = =
                
                title = json_data['o']['w'][0][2]['model']['modules']['TITLE']['mainTitle']['textSpans'][0]['text'].strip()

                # = = = = = = = = = = = = = = =
                
                if 'subTitle' in json_data['o']['w'][0][2]['model']['modules']['TITLE']:
                    subtitle = json_data['o']['w'][0][2]['model']['modules']['TITLE']['subTitle']['textSpans'][0]['text'].strip()
                else:
                    subtitle = ''
                    
                # = = = = = = = = = = = = = = =
                
                if 'binModel' in json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']:
                    price = json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']['binModel']['price']['textSpans'][0]['text'].strip()
                elif 'bidPrice' in json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']:
                    price = json_data['o']['w'][0][2]['model']['modules']['BUY_BOX']['bidPrice']['textSpans'][0]['text'].strip()

                # = = = = = = = = = = = = = = =
                
                if 'QUANTITY' in json_data['o']['w'][0][2]['model']['modules']:
                    available = ''
                    sold = ''
                    list_text = json_data['o']['w'][0][2]['model']['modules']['QUANTITY']['availabilitySignal']['textSpans']
                    for text in list_text:
                        if param_market in ['US', 'AU', 'UK']:
                            if 'available' in text['text']:
                                available = text['text'].strip()
                            elif 'sold' in text['text']:
                                sold = text['text'].strip()
                        elif param_market == 'DE':
                            if 'verfügbar' in text['text']:
                                available = text['text'].strip()
                            elif 'verkauft' in text['text']:
                                sold = text['text'].strip()
                else:
                    available = ''
                    sold = ''
                    
                # = = = = = = = = = = = = = = =
                
                if 'largeImg' in json_data['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']:
                    src = json_data['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']['largeImg']['URL'].strip()
                else:
                    src = json_data['o']['w'][0][2]['model']['modules']['PICTURE']['mediaList'][0]['image']['originalImg']['URL'].strip()

                # = = = = = = = = = = = = = = =
                
                if 'action' in json_data['o']['w'][0][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]:
                    description_url = json_data['o']['w'][0][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['action']['URL'].strip()
                else:
                    description_url = json_data['o']['w'][0][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['dataItems'][0]['action']['URL']

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': '',
                                         'Category_No.': int(input_.loc[a, 'Category_No.']),
                                         'Category': input_.loc[a, 'Category'],
                                         'No.': int(input_.loc[a, 'No.']),
                                         'Item_Number': input_.loc[a, 'Item_Number'],
                                         'Title': title,
                                         'Subtitle': subtitle,
                                         'Price': price,
                                         'Available': available,
                                         'Sold': sold,
                                         'Vehicle_1': '',
                                         'Vehicle_2': '',
                                         'Pic': '',
                                         'Url': url,
                                         'Description_Url': description_url,
                                         'Src': src,
                                         'Vehicle_Page': vehicle_page,
                                         'Category_Id': category_id,
                                         'Market_Id': market_id,
                                         'Country': country,
                                         'Language': language,
                                         'Locale': locale}])

                dict_name = {}
                if param_market in ['US', 'AU', 'UK']:
                    condition = 'Condition'
                elif param_market == 'DE':
                    condition = 'Artikelzustand'

                for name_value in specification:
                    name = specification[name_value]['labels'][0]['textSpans'][0]['text'].strip()
                    if name == condition:
                        df_temp[condition] = specification[name_value]['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip()
                    else:
                        name = name.lower()
                        if name in dict_name:
                            dict_name[name] += 1
                        else:
                            dict_name[name] = 1
                        df_temp[name if dict_name[name] == 1 else name + '-' + str(dict_name[name])] = specification[name_value]['values'][0]['textSpans'][0]['text'].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Category_No.': int(input_.loc[a, 'Category_No.']),
                                     'Category': input_.loc[a, 'Category'],
                                     'No.': int(input_.loc[a, 'No.']),
                                     'Item_Number': input_.loc[a, 'Item_Number']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Category'] + ' - ' + input_.loc[a, 'Item_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Category_No.', 'No.'], ascending=[True, True]).reset_index(drop=True)
output_correct['序号'] = [i+1 for i in range(len(output_correct))]
output_correct.to_excel('./item-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Category_No.', 'No.'], ascending=[True, True]).reset_index(drop=True)
    output_error.to_excel('./item_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：11147

[ok] - Suits Audi - 143535754643
[尝试次数：1] - [剩余数量：11127] - [当前时间：20:24:42]

[ok] - Accessories - 141930238318
[尝试次数：1] - [剩余数量：11126] - [当前时间：20:24:43]

[ok] - Suits Audi - 142126371565
[尝试次数：1] - [剩余数量：11125] - [当前时间：20:24:43]

[ok] - Suits Audi - 142126374045
[尝试次数：1] - [剩余数量：11124] - [当前时间：20:24:43]

[ok] - Accessories - 141974393887
[尝试次数：1] - [剩余数量：11123] - [当前时间：20:24:43]

[ok] - Accessories - 142075267605
[尝试次数：1] - [剩余数量：11122] - [当前时间：20:24:43]

[ok] - Suits Audi - 142575368034
[尝试次数：1] - [剩余数量：11121] - [当前时间：20:24:43]

[ok] - Suits Audi - 142264408326
[尝试次数：1] - [剩余数量：11120] - [当前时间：20:24:44]

[ok] - Accessories - 141974393891
[尝试次数：1] - [剩余数量：11119] - [当前时间：20:24:44]

[ok] - Suits Audi - 142529595152
[尝试次数：1] - [剩余数量：11118] - [当前时间：20:24:44]

[ok] - Suits Audi - 143535755018
[尝试次数：1] - [剩余数量：11117] - [当前时间：20:24:44]

[ok] - Suits Audi - 142815532670
[尝试次数：1] - [剩余数量：11116] - [当前时间：20:24:45]

[ok] - Accessories - 142075267580
[尝试次数：1] - [剩余数量：11115] - [当前时间：20:24:4

[ok] - Suits BMW - 142529587682
[尝试次数：1] - [剩余数量：11019] - [当前时间：20:25:11]

[ok] - Suits BMW - 142475698496
[尝试次数：1] - [剩余数量：11018] - [当前时间：20:25:11]

[ok] - Suits BMW - 142946888102
[尝试次数：1] - [剩余数量：11017] - [当前时间：20:25:11]

[ok] - Suits BMW - 142445113272
[尝试次数：1] - [剩余数量：11016] - [当前时间：20:25:12]

[ok] - Suits BMW - 142333041302
[尝试次数：1] - [剩余数量：11015] - [当前时间：20:25:12]

[ok] - Suits BMW - 142371805636
[尝试次数：1] - [剩余数量：11014] - [当前时间：20:25:12]

[ok] - Suits Audi - 142075055352
[尝试次数：1] - [剩余数量：11013] - [当前时间：20:25:12]

[ok] - Suits BMW - 142328824091
[尝试次数：1] - [剩余数量：11012] - [当前时间：20:25:12]

[ok] - Suits BMW - 142284022291
[尝试次数：1] - [剩余数量：11011] - [当前时间：20:25:13]

[ok] - Suits BMW - 142094894068
[尝试次数：1] - [剩余数量：11010] - [当前时间：20:25:13]

[ok] - Suits Audi - 142684500800
[尝试次数：1] - [剩余数量：11009] - [当前时间：20:25:14]

[ok] - Suits BMW - 142328829975
[尝试次数：1] - [剩余数量：11008] - [当前时间：20:25:14]

[ok] - Suits BMW - 142533368066
[尝试次数：1] - [剩余数量：11007] - [当前时间：20:25:15]

[ok] - Suits BMW - 1420

[ok] - Suits BMW - 142926508992
[尝试次数：1] - [剩余数量：10909] - [当前时间：20:25:35]

[ok] - Suits BMW - 142541833108
[尝试次数：1] - [剩余数量：10908] - [当前时间：20:25:36]

[ok] - Suits BMW - 142585168084
[尝试次数：1] - [剩余数量：10907] - [当前时间：20:25:36]

[ok] - Suits BMW - 142567787080
[尝试次数：1] - [剩余数量：10906] - [当前时间：20:25:37]

[ok] - Suits BMW - 142541833111
[尝试次数：1] - [剩余数量：10905] - [当前时间：20:25:37]

[ok] - Suits BMW - 142075055354
[尝试次数：1] - [剩余数量：10904] - [当前时间：20:25:37]

[ok] - Suits BMW - 142113103204
[尝试次数：1] - [剩余数量：10903] - [当前时间：20:25:37]

[ok] - Suits BMW - 142926509067
[尝试次数：1] - [剩余数量：10902] - [当前时间：20:25:37]

[ok] - Suits BMW - 142577210617
[尝试次数：2] - [剩余数量：10901] - [当前时间：20:25:37]

[ok] - Suits BMW - 142567787180
[尝试次数：1] - [剩余数量：10900] - [当前时间：20:25:37]

[ok] - Suits BMW - 142777199982
[尝试次数：1] - [剩余数量：10899] - [当前时间：20:25:38]

[ok] - Suits BMW - 143235030409
[尝试次数：1] - [剩余数量：10898] - [当前时间：20:25:38]

[ok] - Suits BMW - 142567788416
[尝试次数：1] - [剩余数量：10897] - [当前时间：20:25:39]

[ok] - Suits BMW - 142567

[ok] - Suits Citroen - 142705408236
[尝试次数：1] - [剩余数量：10796] - [当前时间：20:26:01]

[ok] - Suits Citroen - 142736907758
[尝试次数：1] - [剩余数量：10795] - [当前时间：20:26:02]

[ok] - Suits Citroen - 144225772256
[尝试次数：1] - [剩余数量：10794] - [当前时间：20:26:02]

[ok] - Suits Citroen - 142705408217
[尝试次数：1] - [剩余数量：10793] - [当前时间：20:26:02]

[ok] - Suits Citroen - 142736907820
[尝试次数：1] - [剩余数量：10792] - [当前时间：20:26:03]

[ok] - Suits Citroen - 144225772311
[尝试次数：1] - [剩余数量：10791] - [当前时间：20:26:03]

[ok] - Suits Citroen - 142662158706
[尝试次数：1] - [剩余数量：10790] - [当前时间：20:26:03]

[ok] - Suits Daewoo - 142179163061
[尝试次数：1] - [剩余数量：10789] - [当前时间：20:26:03]

[ok] - Suits Daewoo - 142179162883
[尝试次数：1] - [剩余数量：10788] - [当前时间：20:26:04]

[ok] - Suits Fiat - 142475780145
[尝试次数：1] - [剩余数量：10787] - [当前时间：20:26:05]

[ok] - Suits Fiat - 142510932122
[尝试次数：1] - [剩余数量：10786] - [当前时间：20:26:05]

[ok] - Suits Citroen - 142736907759
[尝试次数：1] - [剩余数量：10785] - [当前时间：20:26:05]

[ok] - Suits Fiat - 142475690317
[尝试次数：1] - [剩余数量：10784] - [

[ok] - Suits Ford - 142727195468
[尝试次数：1] - [剩余数量：10687] - [当前时间：20:26:38]

[ok] - Suits Fiat - 142567790169
[尝试次数：1] - [剩余数量：10686] - [当前时间：20:26:38]

[ok] - Suits Ford - 142227136633
[尝试次数：1] - [剩余数量：10685] - [当前时间：20:26:39]

[ok] - Suits Ford - 142291768672
[尝试次数：1] - [剩余数量：10684] - [当前时间：20:26:39]

[ok] - Suits Ford - 142568177348
[尝试次数：1] - [剩余数量：10683] - [当前时间：20:26:39]

[ok] - Suits Ford - 142574312242
[尝试次数：2] - [剩余数量：10682] - [当前时间：20:26:40]

[ok] - Suits Ford - 142094869798
[尝试次数：1] - [剩余数量：10681] - [当前时间：20:26:40]

[ok] - Suits Ford - 142427653221
[尝试次数：1] - [剩余数量：10680] - [当前时间：20:26:41]

[ok] - Suits Ford - 142427653474
[尝试次数：1] - [剩余数量：10679] - [当前时间：20:26:41]

[ok] - Suits Ford - 142567583409
[尝试次数：1] - [剩余数量：10678] - [当前时间：20:26:41]

[ok] - Suits Ford - 142227136591
[尝试次数：1] - [剩余数量：10677] - [当前时间：20:26:41]

[ok] - Suits Ford - 142520209270
[尝试次数：1] - [剩余数量：10676] - [当前时间：20:26:42]

[ok] - Suits Ford - 142980360270
[尝试次数：1] - [剩余数量：10675] - [当前时间：20:26:42]

[ok] - Suits

...

[ok] - Other - 144262998282
[尝试次数：1] - [剩余数量：2423] - [当前时间：21:08:34]

[ok] - Other - 144053491744
[尝试次数：1] - [剩余数量：2422] - [当前时间：21:08:35]

[ok] - Other - 143560785020
[尝试次数：1] - [剩余数量：2421] - [当前时间：21:08:35]

[ok] - Other - 144053491658
[尝试次数：1] - [剩余数量：2420] - [当前时间：21:08:36]

[ok] - Other - 144053491964
[尝试次数：1] - [剩余数量：2419] - [当前时间：21:08:37]

[ok] - Other - 144053491911
[尝试次数：1] - [剩余数量：2418] - [当前时间：21:08:37]

[ok] - Other - 144851530348
[尝试次数：2] - [剩余数量：2417] - [当前时间：21:08:37]

[ok] - Other - 144053502674
[尝试次数：1] - [剩余数量：2416] - [当前时间：21:08:38]

[ok] - Other - 144053502908
[尝试次数：1] - [剩余数量：2415] - [当前时间：21:08:39]

[ok] - Other - 144053502881
[尝试次数：1] - [剩余数量：2414] - [当前时间：21:08:40]

[ok] - Other - 144053491830
[尝试次数：2] - [剩余数量：2413] - [当前时间：21:08:40]

[ok] - Other - 144053502981
[尝试次数：1] - [剩余数量：2412] - [当前时间：21:08:41]

[ok] - Other - 144291289910
[尝试次数：2] - [剩余数量：2411] - [当前时间：21:08:42]

[ok] - Other - 144053505767
[尝试次数：1] - [剩余数量：2410] - [当前时间：21:08:42]

[ok] - Other - 14405

[ok] - Other - 143375103738
[尝试次数：1] - [剩余数量：2305] - [当前时间：21:09:26]

[ok] - Other - 145596606209
[尝试次数：1] - [剩余数量：2304] - [当前时间：21:09:26]

[ok] - Other - 143294185704
[尝试次数：2] - [剩余数量：2303] - [当前时间：21:09:27]

[ok] - Other - 145498002210
[尝试次数：2] - [剩余数量：2302] - [当前时间：21:09:27]

[ok] - Other - 143375104445
[尝试次数：1] - [剩余数量：2301] - [当前时间：21:09:27]

[ok] - Other - 143375104405
[尝试次数：1] - [剩余数量：2300] - [当前时间：21:09:27]

[ok] - Other - 143375105873
[尝试次数：1] - [剩余数量：2299] - [当前时间：21:09:28]

[ok] - Other - 143375105904
[尝试次数：1] - [剩余数量：2298] - [当前时间：21:09:28]

[ok] - Other - 143375102646
[尝试次数：1] - [剩余数量：2297] - [当前时间：21:09:28]

[ok] - Other - 145175592992
[尝试次数：1] - [剩余数量：2296] - [当前时间：21:09:28]

[ok] - Other - 145175657240
[尝试次数：1] - [剩余数量：2295] - [当前时间：21:09:29]

[ok] - Other - 145175656980
[尝试次数：1] - [剩余数量：2294] - [当前时间：21:09:29]

[ok] - Other - 144473525718
[尝试次数：1] - [剩余数量：2293] - [当前时间：21:09:29]

[ok] - Other - 145175657282
[尝试次数：1] - [剩余数量：2292] - [当前时间：21:09:29]

[ok] - Other - 14524

[ok] - Other - 144842299781
[尝试次数：1] - [剩余数量：2186] - [当前时间：21:10:09]

[ok] - Other - 142992739420
[尝试次数：1] - [剩余数量：2185] - [当前时间：21:10:10]

[ok] - Other - 143238318952
[尝试次数：1] - [剩余数量：2184] - [当前时间：21:10:10]

[ok] - Other - 144842299561
[尝试次数：1] - [剩余数量：2183] - [当前时间：21:10:10]

[ok] - Other - 144851531959
[尝试次数：3] - [剩余数量：2182] - [当前时间：21:10:11]

[ok] - Other - 144843835998
[尝试次数：1] - [剩余数量：2181] - [当前时间：21:10:11]

[ok] - Other - 144053588316
[尝试次数：1] - [剩余数量：2180] - [当前时间：21:10:11]

[ok] - Other - 144053588156
[尝试次数：1] - [剩余数量：2179] - [当前时间：21:10:11]

[ok] - Other - 142918251529
[尝试次数：1] - [剩余数量：2178] - [当前时间：21:10:13]

[ok] - Other - 142918252138
[尝试次数：1] - [剩余数量：2177] - [当前时间：21:10:13]

[ok] - Other - 144053589364
[尝试次数：1] - [剩余数量：2176] - [当前时间：21:10:13]

[ok] - Other - 144053589198
[尝试次数：1] - [剩余数量：2175] - [当前时间：21:10:14]

[ok] - Other - 144840787429
[尝试次数：1] - [剩余数量：2174] - [当前时间：21:10:14]

[ok] - Other - 142918251591
[尝试次数：1] - [剩余数量：2173] - [当前时间：21:10:14]

[ok] - Other - 14323

[ok] - Other - 144456283524
[尝试次数：1] - [剩余数量：2068] - [当前时间：21:10:42]

[ok] - Other - 144843572726
[尝试次数：1] - [剩余数量：2067] - [当前时间：21:10:43]

[ok] - Other - 144843570396
[尝试次数：1] - [剩余数量：2066] - [当前时间：21:10:44]

[ok] - Other - 142970719711
[尝试次数：1] - [剩余数量：2065] - [当前时间：21:10:44]

[ok] - Other - 142970716321
[尝试次数：1] - [剩余数量：2064] - [当前时间：21:10:44]

[ok] - Other - 144843573280
[尝试次数：1] - [剩余数量：2063] - [当前时间：21:10:44]

[ok] - Other - 142970721677
[尝试次数：1] - [剩余数量：2062] - [当前时间：21:10:44]

[ok] - Other - 142992737426
[尝试次数：3] - [剩余数量：2061] - [当前时间：21:10:44]

[ok] - Other - 142970724806
[尝试次数：1] - [剩余数量：2060] - [当前时间：21:10:44]

[ok] - Other - 142970722756
[尝试次数：1] - [剩余数量：2059] - [当前时间：21:10:45]

[ok] - Other - 142992730687
[尝试次数：2] - [剩余数量：2058] - [当前时间：21:10:45]

[ok] - Other - 144840788464
[尝试次数：1] - [剩余数量：2057] - [当前时间：21:10:45]

[ok] - Other - 144418479067
[尝试次数：1] - [剩余数量：2056] - [当前时间：21:10:46]

[ok] - Other - 144430037169
[尝试次数：1] - [剩余数量：2055] - [当前时间：21:10:46]

[ok] - Other - 14446

[ok] - Other - 145596540994
[尝试次数：1] - [剩余数量：1950] - [当前时间：21:11:21]

[ok] - Other - 144540863134
[尝试次数：1] - [剩余数量：1949] - [当前时间：21:11:22]

[ok] - Other - 144756007625
[尝试次数：1] - [剩余数量：1948] - [当前时间：21:11:22]

[ok] - Other - 143226414531
[尝试次数：1] - [剩余数量：1947] - [当前时间：21:11:22]

[ok] - Other - 144822500135
[尝试次数：1] - [剩余数量：1946] - [当前时间：21:11:22]

[ok] - Other - 144429583523
[尝试次数：2] - [剩余数量：1945] - [当前时间：21:11:22]

[ok] - Other - 143070419416
[尝试次数：1] - [剩余数量：1944] - [当前时间：21:11:23]

[ok] - Other - 145535955789
[尝试次数：1] - [剩余数量：1943] - [当前时间：21:11:23]

[ok] - Other - 145312275536
[尝试次数：1] - [剩余数量：1942] - [当前时间：21:11:23]

[ok] - Other - 144633106636
[尝试次数：1] - [剩余数量：1941] - [当前时间：21:11:24]

[ok] - Other - 143141424482
[尝试次数：2] - [剩余数量：1940] - [当前时间：21:11:24]

[ok] - Other - 144319738378
[尝试次数：1] - [剩余数量：1939] - [当前时间：21:11:24]

[ok] - Other - 144624160312
[尝试次数：1] - [剩余数量：1938] - [当前时间：21:11:24]

[ok] - Other - 145629435963
[尝试次数：1] - [剩余数量：1937] - [当前时间：21:11:24]

[ok] - Other - 14423